In [19]:
# Import necessary modules
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain


In [20]:
# Define the prompt template
template = """Question: {question}

Answer: Let's work this out in a step by step way to be sure we have the right answer."""
prompt = PromptTemplate.from_template(template)

In [21]:
# Set up callback manager for token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])



In [22]:

# Define Llama model configuration
n_gpu_layers = -1  # The number of layers to put on the GPU
n_batch = 512  # Batch size

# Initialize the Llama model
llm = LlamaCpp(
    model_path="D:\\__repos\\ZZSN24L\\models\\Llama-2-7B-GGUF\\llama-2-7b.Q3_K_M.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)


llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from D:\__repos\ZZSN24L\models\Llama-2-7B-GGUF\llama-2-7b.Q3_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.att

In [23]:
# Create a chain with the prompt and the LLM
llm_chain = LLMChain.invoke(prompt=prompt, llm=llm)


TypeError: Chain.invoke() missing 2 required positional arguments: 'self' and 'input'

In [ ]:

# Function to read and summarize a PDF
def summarize_pdf(pdf_path):
    # Load the PDF
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()

    # Split the text into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)

    # Create a summarize chain
    summarize_chain = load_summarize_chain(llm, chain_type="map_reduce")

    # Get the summary
    summary = summarize_chain.run(texts)
    return summary


In [ ]:

# Example usage
pdf_path = "ZZSN.pdf"
summary = summarize_pdf(pdf_path)
print("Summary of the PDF:", summary)


In [ ]:

# Example question to the LLM
question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"
answer = llm_chain.invoke({"question": question})
print("Answer to the question:", answer)